In [1]:
!pip install pyngrok
!pip install streamlit
!pip install flask_ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [2]:
!ngrok config add-authtoken 2s5IpQU8NpKyTdwUPswhonDqryf_3svrQ8xhDKUFL9NDFNrt5

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify, render_template_string
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
from pyngrok import ngrok

app = Flask(__name__)

# Load the trained model
MODEL_PATH = "/content/Model_InceptionV3.h5"
model = load_model(MODEL_PATH)

# Define constants
IMAGE_RES = 224
class_names = [
    "Alovera", "Amla", "Bamboo", "Beans", "Castor", "Catharanthus",
    "Coffee", "Coriender", "Globe Amarnath", "Jackfruit", "Kasambruga",
    "Lemon", "Mango", "Marigold", "Palak"
]

# Plant uses and locations dictionary
plant_info = {
    "Alovera": {
        "characteristics": "Rich in vitamins C and E; contains antioxidants and anti-inflammatory compounds.",
        "medicinal_uses": [
            "Used topically to heal burns, cuts, and wounds.",
            "Applied for its soothing and moisturizing effects on the skin.",
            "Helps improve digestion when consumed for its detoxifying properties.",
            "Can act as a natural remedy for constipation and digestive discomfort."
        ],
        "famous_in": "Famous in tropical and subtropical regions."
    },
    "Amla": {
        "characteristics": "High in vitamin C and antioxidants; rich in fiber and polyphenols.",
        "medicinal_uses": [
            "Used to boost immunity due to its high vitamin C content.",
            "Helps in digestion and acts as a natural laxative.",
            "Used to enhance skin health and prevent aging.",
            "Commonly used to manage blood sugar levels and support heart health."
        ],
        "famous_in": "Famous in India and Southeast Asia."
    },
    "Bamboo": {
        "characteristics": "Rich in silica and minerals like magnesium and potassium.",
        "medicinal_uses": [
            "Used to improve bone health due to its high silica content.",
            "Bamboo shoots are used in traditional medicine for digestive disorders.",
            "Can be used for treating inflammation and respiratory issues.",
            "Bamboo extracts are sometimes used for detoxification and kidney health."
        ],
        "famous_in": "Famous in East Asia and Southeast Asia."
    },
    "Beans": {
        "characteristics": "High in protein, fiber, and essential vitamins like folate and iron.",
        "medicinal_uses": [
            "Used for improving heart health by lowering cholesterol levels.",
            "Helps regulate blood sugar levels and manage diabetes.",
            "Rich in fiber, which aids in digestive health and gut function.",
            "Used in folk medicine to treat inflammation and support liver health."
        ],
        "famous_in": "Famous worldwide, especially in North and South America."
    },
    "Castor": {
        "characteristics": "Contains ricinoleic acid, which has anti-inflammatory and pain-relieving properties.",
        "medicinal_uses": [
            "Castor oil is used to relieve constipation and promote bowel movement.",
            "Applied topically to reduce inflammation and joint pain.",
            "Used in hair care to promote hair growth and scalp health.",
            "Castor oil has antimicrobial properties, often used for skin infections."
        ],
        "famous_in": "Famous in tropical regions of Asia and Africa."
    },
    "Catharanthus": {
        "characteristics": "Contains alkaloids like vincristine and vinblastine, used in cancer treatment.",
        "medicinal_uses": [
            "Used in traditional medicine for managing diabetes and hypertension.",
            "The plant’s extracts are used to treat cancer, particularly leukemia and lymphoma.",
            "Improves circulation and is used for respiratory issues.",
            "Also used for skin disorders and healing wounds."
        ],
        "famous_in": "Famous in tropical and subtropical regions."
    },
    "Coffee": {
        "characteristics": "High in caffeine, antioxidants, and B-vitamins.",
        "medicinal_uses": [
            "Used to improve mental alertness and cognitive function.",
            "Acts as a natural stimulant for fatigue and mood enhancement.",
            "Rich in antioxidants, which help in reducing the risk of certain diseases.",
            "Helps in digestion and can relieve constipation."
        ],
        "famous_in": "Famous in Latin America, Southeast Asia, and Africa."
    },
    "Coriender": {
        "characteristics": "Rich in vitamins A, C, and K, and contains essential oils with antimicrobial properties.",
        "medicinal_uses": [
            "Used to improve digestion and treat indigestion.",
            "Helps lower blood sugar levels and manage diabetes.",
            "Has anti-inflammatory properties, useful for treating arthritis.",
            "Used in traditional medicine to improve skin conditions like acne."
        ],
        "famous_in": "Famous worldwide, especially in Indian and Mediterranean cuisine."
    },
    "Globe Amarnath": {
        "characteristics": "Rich in antioxidants, protein, and essential amino acids.",
        "medicinal_uses": [
            "Used for its anti-inflammatory and antioxidant properties.",
            "Helps in boosting immunity and fighting infections.",
            "Commonly used to improve digestive health and reduce bloating.",
            "Used in traditional medicine to treat skin conditions and wounds."
        ],
        "famous_in": "Famous in India and South Asia."
    },
    "Jackfruit": {
        "characteristics": "High in vitamins A and C, potassium, and dietary fiber.",
        "medicinal_uses": [
            "Used as a cooling agent in traditional medicine.",
            "Helps in digestion and aids in relieving constipation.",
            "Rich in antioxidants, it supports immune function.",
            "Used for its hydrating properties and to boost skin health."
        ],
        "famous_in": "Famous in tropical regions like Southeast Asia and India."
    },
    "Kasambruga": {
        "characteristics": "Contains alkaloids and other compounds used in traditional medicine.",
        "medicinal_uses": [
            "Used for digestive health, including as a remedy for indigestion and bloating.",
            "Promotes liver health and detoxification.",
            "Has anti-inflammatory properties, useful for treating joint pain.",
            "Used in traditional treatments for kidney health and skin infections."
        ],
        "famous_in": "Famous in Southeast Asia."
    },
    "Lemon": {
        "characteristics": "High in vitamin C, flavonoids, and citric acid.",
        "medicinal_uses": [
            "Used to boost immunity due to its high vitamin C content.",
            "Helps in detoxifying the body and improving digestion.",
            "Has antibacterial properties, useful for treating sore throats.",
            "Used in skincare for its antiseptic and skin-brightening effects."
        ],
        "famous_in": "Famous worldwide, especially in Mediterranean regions."
    },
    "Mango": {
        "characteristics": "Rich in vitamins A, C, and E; contains fiber and antioxidants.",
        "medicinal_uses": [
            "Used to support digestion and gut health due to its fiber content.",
            "Promotes skin health with its antioxidant properties.",
            "Traditionally used to treat heat-related illnesses and dehydration.",
            "Helps in boosting immunity and detoxifying the body."
        ],
        "famous_in": "Famous in tropical and subtropical regions, especially in India."
    },
    "Marigold": {
        "characteristics": "Contains lutein and zeaxanthin, which are beneficial for eye health.",
        "medicinal_uses": [
            "Used in traditional medicine to treat skin irritation and wounds.",
            "Has anti-inflammatory properties and is used for arthritis treatment.",
            "Used in eye care for its beneficial effects on vision.",
            "Applied in the form of teas or extracts to support digestion."
        ],
        "famous_in": "Famous in India, Mexico, and other parts of the world."
    },
    "Palak": {
        "characteristics": "High in vitamins A, C, and K, and rich in iron and folate.",
        "medicinal_uses": [
            "Used to treat anemia due to its high iron content.",
            "Supports digestive health and aids in constipation relief.",
            "Used to lower blood pressure due to its high potassium levels.",
            "Has anti-inflammatory properties, useful for treating pain and swelling."
        ],
        "famous_in": "Famous worldwide, particularly in India and Mediterranean countries."
    }
}
# HTML Template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Plant Image Classification</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background-color: #f1f7f3;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            flex-direction: column;
        }

        h1 {
            color: #3c763d;
            font-size: 3em;
            margin-bottom: 20px;
            text-align: center;
        }
        h2 {
            color: #3c763d;
            margin-bottom: 20px;
            text-align: center;
        }
        #upload-form {
            background-color: #ffffff;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
            max-width: 500px;
            width: 100%;
            text-align: center;
        }

        input[type="file"] {
            font-size: 1.2em;
            padding: 12px;
            border: 2px solid #7bbf6a;
            border-radius: 5px;
            margin-bottom: 20px;
            width: 80%;
            box-sizing: border-box;
            background-color: #eaf5db;
            transition: border 0.3s ease;
        }

        input[type="file"]:hover {
            border-color: #4CAF50;
        }

        button {
            background-color: #4CAF50;
            color: white;
            border: none;
            padding: 12px 25px;
            font-size: 1.2em;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s;
            width: 80%;
            margin-top: 10px;
        }

        button:hover {
            background-color: #45a049;
        }

        #result {
            margin-top: 20px;
            font-size: 1.5em;
            font-weight: bold;
            color: #3c763d;
        }

        #characteristics, #uses, #famous-in, #accuracy {
            margin-top: 15px;
            font-size: 1.1em;
            color: #4d6e45;
            font-weight: normal;
        }

        .image-preview {
            max-width: 200px;
            margin-top: 20px;
            border: 3px solid #7bbf6a;
            border-radius: 8px;
            padding: 10px;
            background-color: #f3f9f3;
        }

        #preview {
            width: 100%;
            height: auto;
            border-radius: 8px;
        }

        .footer {
            margin-top: 40px;
            font-size: 0.9em;
            color: #888;
        }

        .footer a {
            color: #3c763d;
            text-decoration: none;
        }

        .footer a:hover {
            text-decoration: underline;
        }

        /* Responsive design */
        @media (max-width: 768px) {
            h1 {
                font-size: 2.5em;
            }
            h2 {
              font-size: 2em;
            }
            #upload-form {
                padding: 20px;
                width: 90%;
            }

            button {
                font-size: 1.1em;
            }
        }

        @media (max-width: 480px) {
            h1 {
                font-size: 2em;
            }
            h2 {
              font-size: 1.5em;
            }
            input[type="file"], button {
                width: 100%;
                font-size: 1em;
            }

            #result {
                font-size: 1.2em;
            }

            #characteristics, #uses, #famous-in {
                font-size: 1em;
            }
        }
    </style>
</head>
<body>
    <h1>Deep Learning based Robust Medicinal Leaves Identification</h1>
    <h2> Upload plant leaf image</h2>
    <form id="upload-form" enctype="multipart/form-data">
        <input type="file" id="image-input" accept="image/*" required>
        <button type="submit">Classify Image</button>
    </form>
    <div id="result"></div>
    <div id="accuracy"></div>
    <div id="characteristics"></div>
    <div id="uses"></div>
    <div id="famous-in"></div>
    <div class="image-preview">
        <img id="preview" src="" alt="" style="display: none;">
    </div>

    <script>
        const form = document.getElementById("upload-form");
        const imageInput = document.getElementById("image-input");
        const resultDiv = document.getElementById("result");
        const characteristicsDiv = document.getElementById("characteristics");
        const usesDiv = document.getElementById("uses");
        const famousInDiv = document.getElementById("famous-in");
        const previewImg = document.getElementById("preview");
        const accuracyDiv = document.getElementById("accuracy");
        form.addEventListener("submit", async (event) => {
            event.preventDefault();

            // Display image preview
            const file = imageInput.files[0];
            const reader = new FileReader();
            reader.onload = () => {
                previewImg.src = reader.result;
                previewImg.style.display = "block";
            };
            reader.readAsDataURL(file);

            // Prepare form data
            const formData = new FormData();
            formData.append("file", file);

            // Send request to the backend
            resultDiv.textContent = "Classifying...";
            try {
                const response = await fetch("/predict", {
                    method: "POST",
                    body: formData,
                });

                if (!response.ok) {
                    throw new Error("Failed to classify the image.");
                }

                const data = await response.json();
                resultDiv.textContent = `Predicted Class: ${data.predicted_class}`;
                characteristicsDiv.textContent = `Characteristics: ${data.characteristics}`;
                usesDiv.textContent = `Uses: ${data.uses}`;
                famousInDiv.textContent = `Famous in: ${data.famous_in}`;
                accuracyDiv.textContent = `Accuracy Score : ${(data.accuracy * 100).toFixed(2)}%`;

            } catch (error) {
                resultDiv.textContent = "Error: " + error.message;
            }
        });
    </script>
</body>
</html>

"""

@app.route("/")
def home():
    return render_template_string(html_template)

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No file selected"}), 400
    temp_path = os.path.join("uploads", file.filename)
    os.makedirs("uploads", exist_ok=True)
    file.save(temp_path)
    input_image = load_img(temp_path, target_size=(IMAGE_RES, IMAGE_RES))
    input_image_array = img_to_array(input_image) / 255.0
    input_image_array = input_image_array[np.newaxis, ...]
    predictions = model.predict(input_image_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class_index]
    accuracy = np.max(predictions)

    # Get plant uses and famous location
    plant_details = plant_info.get(predicted_class_name,{"characteristics": "N/A", "uses": "N/A", "famous_in": "N/A"})

    os.remove(temp_path)
    return jsonify({
        "predicted_class": predicted_class_name,
        "characteristics": plant_details["characteristics"],
        "uses": plant_details["medicinal_uses"],
        "famous_in": plant_details["famous_in"],
        "accuracy": float(accuracy)

    })

if __name__ == "__main__":
    # Open a public URL via ngrok
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(port=5000)


Public URL: NgrokTunnel: "https://acfc-34-75-164-30.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 03:01:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 03:01:36] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 03:02:42] "POST /predict HTTP/1.1" 200 -
